In [ ]:
!pip install selenium
!pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import re

## Скачивание книг в формате .fb2

In [ ]:
def get_pages_with_books(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    books_links = soup.find_all('a', class_='text-decoration-none')
    books_urls = [link['href'] for link in books_links if ('href' in link.attrs and re.match(r"^https://litlife\.club/books/\d+$", link['href']))]
    return books_urls

In [ ]:
def download_fb2_file(page_url, download_dir):
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": download_dir}
    options.add_experimental_option("prefs", prefs)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    try:
        driver.get(page_url)
        wait = WebDriverWait(driver, 10)
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '.fb2.zip')]")))
        download_button.click()
        time.sleep(15)
        print(f"Файл должен быть загружен в: {download_dir}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")
    finally:
        driver.quit()

In [ ]:
download_dir = "data"
count_of_pages = 20
for x in range(count_of_pages):
    main_page_url = f"https://litlife.club/genres/56-klassicheskaya-proza?page={x}"
    books_url = get_pages_with_books(main_page_url)
    for book in books_url:
        download_fb2_file(book, download_dir)

## Преоброзование файлов формата .fb2 в .txt

In [ ]:
all_files = os.listdir("data")
for file_zip in all_files:
    with zipfile.ZipFile(f"data/{file_zip}", 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith(".fb2"):
                try:
                    fb2_file = zip_ref.open(file)
                    fb2_content = fb2_file.read()
                    text_content = ''
                    soup = BeautifulSoup(fb2_content, 'lxml-xml')
                    for para in soup.find_all(['p', 'section']):
                        text_content += para.get_text(separator='\n', strip=True) + '\n'

                    with open(f"books_txt/{file}.txt", 'w', encoding='utf-8') as txt_file:
                        txt_file.write(text_content)
                except FileNotFoundError:
                    ...